In [2]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time
import re
from bs4 import BeautifulSoup 
import random 
import multiprocessing
from tqdm import tqdm



# driver = webdriver.Chrome(executable_path=r'C:\\Users\\ui\\Desktop\chromedriver.exe') # 웹드라이버가 설치된 경로를 지정해주시면 됩니다.

In [ ]:
df = pd.read_csv('./제주특별자치도음식점목록(통합).csv', sep=',', encoding='CP949')
df['naver_map_url'] = '' # 미리 url을 담을 column을 만들어줌

chromedriver = './chromedriver' 
driver = webdriver.Chrome(chromedriver) 


In [38]:
df

,사업장명,업종구분대분류,업종구분소분류,인허가일자,인허가취소일자,영업상태명,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,재개업일자,소재지면적,소재지전체주소,도로명전체주소,도로명우편번호,데이터갱신일자,naver_map_url
0,카페송키,일반음식점,기타,20181129,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,80.14,제주특별자치도 제주시 삼도이동 1120-2번지,"제주특별자치도 제주시 무근성안길 16, 1층 (삼도이동)",63166.0,2018-12-01 2:20,
1,그림책방앤카페노란우산,일반음식점,기타,20181227,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,104.25,제주특별자치도 제주시 애월읍 광령리 2981-1번지 자연숲,"제주특별자치도 제주시 애월읍 하광로 515, 104동 102호 (자연숲)",63062.0,2019-01-27 2:40,
2,에릭스에스프레소,일반음식점,기타,20181102,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,27,제주특별자치도 제주시 구좌읍 세화리 1397-6번지,"제주특별자치도 제주시 구좌읍 구좌로 77, 1층",63361.0,2018-11-04 2:36,
3,일품순두부한림점,일반음식점,한식,20190107,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,152,제주특별자치도 제주시 한림읍 대림리 1845-2번지,"제주특별자치도 제주시 한림읍 한림상로 237, 1층",63030.0,2019-01-17 2:40,
4,팔도실비집아라점,일반음식점,식육(숯불구이),20190107,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,82,제주특별자치도 제주시 아라일동 6139,"제주특별자치도 제주시 중앙로 581, 에이동 1층 (아라일동)",63248.0,2020-09-13 2:40,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42597,커피전문점,휴게음식점,푸드트럭,20181214,NaN,폐업,폐업,20201130.0,NaN,NaN,NaN,8.89,제주특별자치도 서귀포시 동홍동 2142-7 도로,NaN,NaN,2020-12-12 2:40,
42598,아름다운서귀포,휴게음식점,푸드트럭,20181112,NaN,폐업,폐업,20201031.0,NaN,NaN,NaN,12.21,제주특별자치도 서귀포시 서홍동 696 앞도로,NaN,NaN,2020-11-20 2:40,
42599,붕어빵과오뎅,휴게음식점,기타 휴게음식점,20181123,NaN,폐업,폐업,20201202.0,NaN,NaN,NaN,6.48,제주특별자치도 서귀포시 성산읍 고성리 1197-7 생고기부위별판매장,"제주특별자치도 서귀포시 성산읍 고성동서로56번길 15, 생고기부위별판매장",63640.0,2020-12-04 2:40,
42600,인정김밥,휴게음식점,일반조리판매,20210121,NaN,폐업,폐업,20210524.0,NaN,NaN,NaN,35,제주특별자치도 서귀포시 안덕면 서광리 1412,"제주특별자치도 서귀포시 안덕면 녹차분재로 56, 다동",63521.0,2021-05-26 2:40,


In [39]:
# 폐업한 가게는 삭제
df = df[df.영업상태명 != '폐업']

In [40]:
df = df.reset_index()

In [41]:
del df['index']

In [42]:
# 데이터프레임 전체를 1/n으로 슬라이싱하여 크롤링

df = df.iloc[9576:12768]
# df = df.iloc[12768:15961]

In [43]:
df = df.reset_index()
del df['index']

In [44]:
df

,사업장명,업종구분대분류,업종구분소분류,인허가일자,인허가취소일자,영업상태명,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,재개업일자,소재지면적,소재지전체주소,도로명전체주소,도로명우편번호,데이터갱신일자,naver_map_url
0,두마리찜닭두찜앤떡참제주함덕점,일반음식점,한식,20210402,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,88,제주특별자치도 제주시 조천읍 함덕리 4157-3,"제주특별자치도 제주시 조천읍 함와로 21, 1층",63333.0,2021-04-04 0:23,
1,스칼렛가든,일반음식점,뷔페식,20210402,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,217.49,제주특별자치도 제주시 연동 281-22,"제주특별자치도 제주시 삼무로 57, 3층 (연동)",63125.0,2021-04-04 0:23,
2,별도봉오리사냥,일반음식점,한식,20210415,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,68.52,제주특별자치도 제주시 화북일동 4595-2,"제주특별자치도 제주시 별도봉길 7, 1층 (화북일동)",63293.0,2021-06-26 2:40,
3,오조파스타앤피자,일반음식점,기타,20210415,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,137.28,제주특별자치도 제주시 구좌읍 상도리 471-58,"제주특별자치도 제주시 구좌읍 해녀박물관길 30, 1층",63362.0,2021-05-23 2:40,
4,맛있는밥집,일반음식점,한식,20210415,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,99.4,제주특별자치도 제주시 애월읍 광령리 4056-1,"제주특별자치도 제주시 애월읍 하광로 392, 1층 1호",63062.0,2021-04-17 0:22,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3187,(주)코앤엠롯데스카이힐제주CC스타트하우스,일반음식점,중국식,20160408,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,99.22,제주특별자치도 서귀포시 색달동 100,"제주특별자치도 서귀포시 상예로 530, 1층 (색달동)",63549.0,2020-12-20 2:40,
3188,서광보말칼국수,일반음식점,한식,20160411,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,108.17,제주특별자치도 서귀포시 안덕면 서광리 716-1번지,제주특별자치도 서귀포시 안덕면 중산간서로 1882,63522.0,2018-08-31 23:59,
3189,창꼬,일반음식점,경양식,20160411,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,56.8,제주특별자치도 서귀포시 대정읍 신평리 370-2번지,제주특별자치도 서귀포시 대정읍 중산간서로 2598,63516.0,2018-08-31 23:59,
3190,어느멋진날서귀포점,일반음식점,일식,20160412,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,197.69,제주특별자치도 서귀포시 동홍동 412-6,"제주특별자치도 서귀포시 동홍중앙로 68, 1층 (동홍동)",63587.0,2021-06-04 2:40,


In [45]:
# 도로명주소를 , 기준으로 split()하여 사용함 (검색결과의 정확성 높이기 위해)

address_list = df.도로명전체주소.str.split(',').str[0].tolist()
print(address_list)

['제주특별자치도 제주시 조천읍 함와로 21', '제주특별자치도 제주시 삼무로 57', '제주특별자치도 제주시 별도봉길 7', '제주특별자치도 제주시 구좌읍 해녀박물관길 30', '제주특별자치도 제주시 애월읍 하광로 392', '제주특별자치도 제주시 구좌읍 해맞이해안로 1460', '제주특별자치도 제주시 아란9길 22', '제주특별자치도 제주시 아연로 306', '제주특별자치도 제주시 한림읍 명랑로 16-1', '제주특별자치도 제주시 중앙로15길 16', '제주특별자치도 제주시 조천읍 북촌9길 31', '제주특별자치도 제주시 구좌읍 비자숲길 62', '제주특별자치도 제주시 애월읍 애월북서길 56-1', '제주특별자치도 제주시 한림읍 귀덕5길 33', '제주특별자치도 제주시 신설로9길 26-12', '제주특별자치도 제주시 한경면 노을해안로 1160', '제주특별자치도 제주시 노형로 121', '제주특별자치도 제주시 서광로2길 11-24', '제주특별자치도 제주시 애월읍 애월로 12', '제주특별자치도 제주시 구좌읍 해맞이해안로 2484', '제주특별자치도 제주시 아란7길 3-3', '제주특별자치도 제주시 서광로20길 31', '제주특별자치도 제주시 한경면 고산로 33-3', '제주특별자치도 제주시 용남1길 16', '제주특별자치도 제주시 탑동로 31-3', '제주특별자치도 제주시 서해안로 364', '제주특별자치도 제주시 한림읍 한림해안로 279', '제주특별자치도 제주시 노형로 21', '제주특별자치도 제주시 원남6길 54', '제주특별자치도 제주시 광양10길 5', '제주특별자치도 제주시 성신로1길 23', '제주특별자치도 제주시 애월읍 애월해안로 927', '제주특별자치도 제주시 첨단로 199', '제주특별자치도 제주시 천수로8길 1', '제주특별자치도 제주시 한경면 판포4길 23', '제주특별자치도 제주시 관덕로 59', '제주특별자치도 제주시 동광로1길 31', '제주특별자치도 제주시 한경면 조수2길 25', '제주특별자치도 제주시 원노형7길 3', '

In [46]:
df['주소'] = address_list
df

,사업장명,업종구분대분류,업종구분소분류,인허가일자,인허가취소일자,영업상태명,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,재개업일자,소재지면적,소재지전체주소,도로명전체주소,도로명우편번호,데이터갱신일자,naver_map_url,주소
0,두마리찜닭두찜앤떡참제주함덕점,일반음식점,한식,20210402,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,88,제주특별자치도 제주시 조천읍 함덕리 4157-3,"제주특별자치도 제주시 조천읍 함와로 21, 1층",63333.0,2021-04-04 0:23,,제주특별자치도 제주시 조천읍 함와로 21
1,스칼렛가든,일반음식점,뷔페식,20210402,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,217.49,제주특별자치도 제주시 연동 281-22,"제주특별자치도 제주시 삼무로 57, 3층 (연동)",63125.0,2021-04-04 0:23,,제주특별자치도 제주시 삼무로 57
2,별도봉오리사냥,일반음식점,한식,20210415,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,68.52,제주특별자치도 제주시 화북일동 4595-2,"제주특별자치도 제주시 별도봉길 7, 1층 (화북일동)",63293.0,2021-06-26 2:40,,제주특별자치도 제주시 별도봉길 7
3,오조파스타앤피자,일반음식점,기타,20210415,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,137.28,제주특별자치도 제주시 구좌읍 상도리 471-58,"제주특별자치도 제주시 구좌읍 해녀박물관길 30, 1층",63362.0,2021-05-23 2:40,,제주특별자치도 제주시 구좌읍 해녀박물관길 30
4,맛있는밥집,일반음식점,한식,20210415,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,99.4,제주특별자치도 제주시 애월읍 광령리 4056-1,"제주특별자치도 제주시 애월읍 하광로 392, 1층 1호",63062.0,2021-04-17 0:22,,제주특별자치도 제주시 애월읍 하광로 392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3187,(주)코앤엠롯데스카이힐제주CC스타트하우스,일반음식점,중국식,20160408,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,99.22,제주특별자치도 서귀포시 색달동 100,"제주특별자치도 서귀포시 상예로 530, 1층 (색달동)",63549.0,2020-12-20 2:40,,제주특별자치도 서귀포시 상예로 530
3188,서광보말칼국수,일반음식점,한식,20160411,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,108.17,제주특별자치도 서귀포시 안덕면 서광리 716-1번지,제주특별자치도 서귀포시 안덕면 중산간서로 1882,63522.0,2018-08-31 23:59,,제주특별자치도 서귀포시 안덕면 중산간서로 1882
3189,창꼬,일반음식점,경양식,20160411,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,56.8,제주특별자치도 서귀포시 대정읍 신평리 370-2번지,제주특별자치도 서귀포시 대정읍 중산간서로 2598,63516.0,2018-08-31 23:59,,제주특별자치도 서귀포시 대정읍 중산간서로 2598
3190,어느멋진날서귀포점,일반음식점,일식,20160412,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,197.69,제주특별자치도 서귀포시 동홍동 412-6,"제주특별자치도 서귀포시 동홍중앙로 68, 1층 (동홍동)",63587.0,2021-06-04 2:40,,제주특별자치도 서귀포시 동홍중앙로 68


In [47]:
# 정정코드
# 사업장명 + 도로명주소 = '사업장명add'
# 지번주소는 정확하지 않은 주소가 많아서 도로명 주소 사용함

df["사업장명add"]=df['사업장명']+""+df['주소'] 

In [48]:
# 정정한 코드

df = df[['사업장명','사업장명add', '업종구분대분류', '업종구분소분류', '주소', 'naver_map_url']]

In [49]:
df

,사업장명,사업장명add,업종구분대분류,업종구분소분류,주소,naver_map_url
0,두마리찜닭두찜앤떡참제주함덕점,두마리찜닭두찜앤떡참제주함덕점제주특별자치도 제주시 조천읍 함와로 21,일반음식점,한식,제주특별자치도 제주시 조천읍 함와로 21,
1,스칼렛가든,스칼렛가든제주특별자치도 제주시 삼무로 57,일반음식점,뷔페식,제주특별자치도 제주시 삼무로 57,
2,별도봉오리사냥,별도봉오리사냥제주특별자치도 제주시 별도봉길 7,일반음식점,한식,제주특별자치도 제주시 별도봉길 7,
3,오조파스타앤피자,오조파스타앤피자제주특별자치도 제주시 구좌읍 해녀박물관길 30,일반음식점,기타,제주특별자치도 제주시 구좌읍 해녀박물관길 30,
4,맛있는밥집,맛있는밥집제주특별자치도 제주시 애월읍 하광로 392,일반음식점,한식,제주특별자치도 제주시 애월읍 하광로 392,
...,...,...,...,...,...,...
3187,(주)코앤엠롯데스카이힐제주CC스타트하우스,(주)코앤엠롯데스카이힐제주CC스타트하우스제주특별자치도 서귀포시 상예로 530,일반음식점,중국식,제주특별자치도 서귀포시 상예로 530,
3188,서광보말칼국수,서광보말칼국수제주특별자치도 서귀포시 안덕면 중산간서로 1882,일반음식점,한식,제주특별자치도 서귀포시 안덕면 중산간서로 1882,
3189,창꼬,창꼬제주특별자치도 서귀포시 대정읍 중산간서로 2598,일반음식점,경양식,제주특별자치도 서귀포시 대정읍 중산간서로 2598,
3190,어느멋진날서귀포점,어느멋진날서귀포점제주특별자치도 서귀포시 동홍중앙로 68,일반음식점,일식,제주특별자치도 서귀포시 동홍중앙로 68,


In [51]:
for i, keyword in enumerate(df['사업장명add'].tolist()):  #정정코드
    print("이번에 찾을 키워드 :", i, f"/ {df.shape[0]} 행",keyword)
    
    try:
        naver_map_search_url = f"https://map.naver.com/v5/search/{keyword}/place" # 검색 url 만들기
        driver.get(naver_map_search_url) # 검색 url 접속, 즉 검색하기
        
        time.sleep(2) # 중요함
        cu = driver.current_url # 검색이 성공된 플레이스에 대한 개별 페이지
        
        res_code = re.findall(r"place/(\d+)", cu)
        final_url = 'https://pcmap.place.naver.com/restaurant/'+res_code[0]+'/review/visitor#'
        
        print(final_url)
        df['naver_map_url'][i]=final_url
        
    except IndexError:
        df['naver_map_url'][i]= ''
        print('none')

이번에 찾을 키워드 : 0 / 3192 행 두마리찜닭두찜앤떡참제주함덕점제주특별자치도 제주시 조천읍 함와로 21


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


none
이번에 찾을 키워드 : 1 / 3192 행 스칼렛가든제주특별자치도 제주시 삼무로 57
none
이번에 찾을 키워드 : 2 / 3192 행 별도봉오리사냥제주특별자치도 제주시 별도봉길 7
https://pcmap.place.naver.com/restaurant/1175295311/review/visitor#
이번에 찾을 키워드 : 3 / 3192 행 오조파스타앤피자제주특별자치도 제주시 구좌읍 해녀박물관길 30
https://pcmap.place.naver.com/restaurant/36882244/review/visitor#
이번에 찾을 키워드 : 4 / 3192 행 맛있는밥집제주특별자치도 제주시 애월읍 하광로 392
https://pcmap.place.naver.com/restaurant/1938752885/review/visitor#
이번에 찾을 키워드 : 5 / 3192 행 구슬함박제주특별자치도 제주시 구좌읍 해맞이해안로 1460
https://pcmap.place.naver.com/restaurant/1707615714/review/visitor#
이번에 찾을 키워드 : 6 / 3192 행 낭만상사제주특별자치도 제주시 아란9길 22
https://pcmap.place.naver.com/restaurant/1090512551/review/visitor#
이번에 찾을 키워드 : 7 / 3192 행 소원이제주특별자치도 제주시 아연로 306
https://pcmap.place.naver.com/restaurant/1833455779/review/visitor#
이번에 찾을 키워드 : 8 / 3192 행 푸라닭제주한림협재점제주특별자치도 제주시 한림읍 명랑로 16-1
https://pcmap.place.naver.com/restaurant/1567792687/review/visitor#
이번에 찾을 키워드 : 9 / 3192 행 잘도식당제주특별자치도 제주시 중앙로15길 16
https://pcmap.place.naver.com/restaurant/147786

# 데이터프레임 행이 1653 개 이상 나오면 코드 수정 성공 😜

In [52]:
df

,사업장명,사업장명add,업종구분대분류,업종구분소분류,주소,naver_map_url
0,두마리찜닭두찜앤떡참제주함덕점,두마리찜닭두찜앤떡참제주함덕점제주특별자치도 제주시 조천읍 함와로 21,일반음식점,한식,제주특별자치도 제주시 조천읍 함와로 21,
1,스칼렛가든,스칼렛가든제주특별자치도 제주시 삼무로 57,일반음식점,뷔페식,제주특별자치도 제주시 삼무로 57,
2,별도봉오리사냥,별도봉오리사냥제주특별자치도 제주시 별도봉길 7,일반음식점,한식,제주특별자치도 제주시 별도봉길 7,https://pcmap.place.naver.com/restaurant/11752...
3,오조파스타앤피자,오조파스타앤피자제주특별자치도 제주시 구좌읍 해녀박물관길 30,일반음식점,기타,제주특별자치도 제주시 구좌읍 해녀박물관길 30,https://pcmap.place.naver.com/restaurant/36882...
4,맛있는밥집,맛있는밥집제주특별자치도 제주시 애월읍 하광로 392,일반음식점,한식,제주특별자치도 제주시 애월읍 하광로 392,https://pcmap.place.naver.com/restaurant/19387...
...,...,...,...,...,...,...
3187,(주)코앤엠롯데스카이힐제주CC스타트하우스,(주)코앤엠롯데스카이힐제주CC스타트하우스제주특별자치도 서귀포시 상예로 530,일반음식점,중국식,제주특별자치도 서귀포시 상예로 530,
3188,서광보말칼국수,서광보말칼국수제주특별자치도 서귀포시 안덕면 중산간서로 1882,일반음식점,한식,제주특별자치도 서귀포시 안덕면 중산간서로 1882,https://pcmap.place.naver.com/restaurant/18247...
3189,창꼬,창꼬제주특별자치도 서귀포시 대정읍 중산간서로 2598,일반음식점,경양식,제주특별자치도 서귀포시 대정읍 중산간서로 2598,https://pcmap.place.naver.com/restaurant/37840...
3190,어느멋진날서귀포점,어느멋진날서귀포점제주특별자치도 서귀포시 동홍중앙로 68,일반음식점,일식,제주특별자치도 서귀포시 동홍중앙로 68,


In [55]:
# df
df['naver_map_url'].replace('', np.nan, inplace=True)

In [56]:
df = df.dropna(subset=['naver_map_url'])

In [57]:
df

,사업장명,사업장명add,업종구분대분류,업종구분소분류,주소,naver_map_url
2,별도봉오리사냥,별도봉오리사냥제주특별자치도 제주시 별도봉길 7,일반음식점,한식,제주특별자치도 제주시 별도봉길 7,https://pcmap.place.naver.com/restaurant/11752...
3,오조파스타앤피자,오조파스타앤피자제주특별자치도 제주시 구좌읍 해녀박물관길 30,일반음식점,기타,제주특별자치도 제주시 구좌읍 해녀박물관길 30,https://pcmap.place.naver.com/restaurant/36882...
4,맛있는밥집,맛있는밥집제주특별자치도 제주시 애월읍 하광로 392,일반음식점,한식,제주특별자치도 제주시 애월읍 하광로 392,https://pcmap.place.naver.com/restaurant/19387...
5,구슬함박,구슬함박제주특별자치도 제주시 구좌읍 해맞이해안로 1460,일반음식점,경양식,제주특별자치도 제주시 구좌읍 해맞이해안로 1460,https://pcmap.place.naver.com/restaurant/17076...
6,낭만상사,낭만상사제주특별자치도 제주시 아란9길 22,일반음식점,분식,제주특별자치도 제주시 아란9길 22,https://pcmap.place.naver.com/restaurant/10905...
...,...,...,...,...,...,...
3183,낙지마당,낙지마당제주특별자치도 서귀포시 신중로 73 (서호동),일반음식점,한식,제주특별자치도 서귀포시 신중로 73 (서호동),https://pcmap.place.naver.com/restaurant/16902...
3184,화룡양꼬치,화룡양꼬치제주특별자치도 서귀포시 이중섭로 12 (서귀동),일반음식점,한식,제주특별자치도 서귀포시 이중섭로 12 (서귀동),https://pcmap.place.naver.com/restaurant/31697...
3186,청해수산회센터,청해수산회센터제주특별자치도 서귀포시 남원읍 남태해안로 211,일반음식점,회집,제주특별자치도 서귀포시 남원읍 남태해안로 211,https://pcmap.place.naver.com/restaurant/38402...
3188,서광보말칼국수,서광보말칼국수제주특별자치도 서귀포시 안덕면 중산간서로 1882,일반음식점,한식,제주특별자치도 서귀포시 안덕면 중산간서로 1882,https://pcmap.place.naver.com/restaurant/18247...


In [58]:
df = df.reset_index()
df

,index,사업장명,사업장명add,업종구분대분류,업종구분소분류,주소,naver_map_url
0,2,별도봉오리사냥,별도봉오리사냥제주특별자치도 제주시 별도봉길 7,일반음식점,한식,제주특별자치도 제주시 별도봉길 7,https://pcmap.place.naver.com/restaurant/11752...
1,3,오조파스타앤피자,오조파스타앤피자제주특별자치도 제주시 구좌읍 해녀박물관길 30,일반음식점,기타,제주특별자치도 제주시 구좌읍 해녀박물관길 30,https://pcmap.place.naver.com/restaurant/36882...
2,4,맛있는밥집,맛있는밥집제주특별자치도 제주시 애월읍 하광로 392,일반음식점,한식,제주특별자치도 제주시 애월읍 하광로 392,https://pcmap.place.naver.com/restaurant/19387...
3,5,구슬함박,구슬함박제주특별자치도 제주시 구좌읍 해맞이해안로 1460,일반음식점,경양식,제주특별자치도 제주시 구좌읍 해맞이해안로 1460,https://pcmap.place.naver.com/restaurant/17076...
4,6,낭만상사,낭만상사제주특별자치도 제주시 아란9길 22,일반음식점,분식,제주특별자치도 제주시 아란9길 22,https://pcmap.place.naver.com/restaurant/10905...
...,...,...,...,...,...,...,...
2131,3183,낙지마당,낙지마당제주특별자치도 서귀포시 신중로 73 (서호동),일반음식점,한식,제주특별자치도 서귀포시 신중로 73 (서호동),https://pcmap.place.naver.com/restaurant/16902...
2132,3184,화룡양꼬치,화룡양꼬치제주특별자치도 서귀포시 이중섭로 12 (서귀동),일반음식점,한식,제주특별자치도 서귀포시 이중섭로 12 (서귀동),https://pcmap.place.naver.com/restaurant/31697...
2133,3186,청해수산회센터,청해수산회센터제주특별자치도 서귀포시 남원읍 남태해안로 211,일반음식점,회집,제주특별자치도 서귀포시 남원읍 남태해안로 211,https://pcmap.place.naver.com/restaurant/38402...
2134,3188,서광보말칼국수,서광보말칼국수제주특별자치도 서귀포시 안덕면 중산간서로 1882,일반음식점,한식,제주특별자치도 서귀포시 안덕면 중산간서로 1882,https://pcmap.place.naver.com/restaurant/18247...


In [59]:
del df['index']

In [60]:
df

,사업장명,사업장명add,업종구분대분류,업종구분소분류,주소,naver_map_url
0,별도봉오리사냥,별도봉오리사냥제주특별자치도 제주시 별도봉길 7,일반음식점,한식,제주특별자치도 제주시 별도봉길 7,https://pcmap.place.naver.com/restaurant/11752...
1,오조파스타앤피자,오조파스타앤피자제주특별자치도 제주시 구좌읍 해녀박물관길 30,일반음식점,기타,제주특별자치도 제주시 구좌읍 해녀박물관길 30,https://pcmap.place.naver.com/restaurant/36882...
2,맛있는밥집,맛있는밥집제주특별자치도 제주시 애월읍 하광로 392,일반음식점,한식,제주특별자치도 제주시 애월읍 하광로 392,https://pcmap.place.naver.com/restaurant/19387...
3,구슬함박,구슬함박제주특별자치도 제주시 구좌읍 해맞이해안로 1460,일반음식점,경양식,제주특별자치도 제주시 구좌읍 해맞이해안로 1460,https://pcmap.place.naver.com/restaurant/17076...
4,낭만상사,낭만상사제주특별자치도 제주시 아란9길 22,일반음식점,분식,제주특별자치도 제주시 아란9길 22,https://pcmap.place.naver.com/restaurant/10905...
...,...,...,...,...,...,...
2131,낙지마당,낙지마당제주특별자치도 서귀포시 신중로 73 (서호동),일반음식점,한식,제주특별자치도 서귀포시 신중로 73 (서호동),https://pcmap.place.naver.com/restaurant/16902...
2132,화룡양꼬치,화룡양꼬치제주특별자치도 서귀포시 이중섭로 12 (서귀동),일반음식점,한식,제주특별자치도 서귀포시 이중섭로 12 (서귀동),https://pcmap.place.naver.com/restaurant/31697...
2133,청해수산회센터,청해수산회센터제주특별자치도 서귀포시 남원읍 남태해안로 211,일반음식점,회집,제주특별자치도 서귀포시 남원읍 남태해안로 211,https://pcmap.place.naver.com/restaurant/38402...
2134,서광보말칼국수,서광보말칼국수제주특별자치도 서귀포시 안덕면 중산간서로 1882,일반음식점,한식,제주특별자치도 서귀포시 안덕면 중산간서로 1882,https://pcmap.place.naver.com/restaurant/18247...


In [61]:
df.to_csv('url_completed.csv', encoding = 'utf-8-sig')

In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time
import re

from bs4 import BeautifulSoup
from tqdm import tqdm

# 웹드라이버 접속
driver = webdriver.Chrome(executable_path='./chromedriver.exe')

df = pd.read_csv('url_completed.csv') #null값 처리 완료 데이터

# 수집할 정보들
rating_list = [] # 평점
user_review_id = {} # 유저의 id - 추천시스템에서는 필수
review_json = {} # 리뷰
image_json = {} # 이미지

In [ ]:
# for i in range(0, 200): # 1회차: len(df)
# for i in range(201, 400):
for i in range(401, 600):
# for i in range(601, 800):
# for i in range(801, 1000):
# for i in range(1001, 1200):
# for i in range(1201, 1400):
# for i in range(1401, 1600):
# for i in range(1601, 1800):
# for i in range(1801, 2000):
# for i in range(2001, (len(df)+1)):

    print('======================================================')
    print(str(i)+'번째 식당')
    
    # 식당 리뷰 개별 url 접속
    driver.get(df['naver_map_url'][i])
    thisurl = df['naver_map_url'][i]
    time.sleep(2)
    # 더보기 버튼 다 누르기
    # 더보기 버튼 누르기까지 10개
    # 더보기 버튼 누르면 10개 추가됨
    while True:
        try:
            time.sleep(1) # 1초로 바꾸고
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(3.5) # 4초로 바꾸고
            driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div[2]/div[5]/div[4]/div[5]/div[2]/a').click()
            time.sleep(3.5) # 4초로 바꾸고
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(1)  # 1초로 바꾸고
            
        except NoSuchElementException:
            print('-더보기 버튼 모두 클릭 완료-')
            break

    
    # 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    time.sleep(1)
    
    # 식당 구분
    restaurant_name = df['사업장명'][i]
    print('식당 이름 : '+restaurant_name)
    
    user_review_id[restaurant_name] = {}
    review_json[restaurant_name] = {}
    image_json[restaurant_name] = {}
    
    try:
        restaurant_classificaton = soup.find_all('span',attrs = {'class':'_3ocDE'})[0].text
    except:
        restaurant_classificaton = 'none'
    print('식당 구분 : '+restaurant_classificaton)
    print('----------------------------------------------')
    
    try:
        one_review = soup.find_all('div', attrs = {'class':'_1Z_GL'})
        review_num = len(one_review) # 특정 식당의 리뷰 총 개수
        print('리뷰 총 개수 : '+str(review_num))
    
        # 리뷰 개수
        for i in range(len(one_review)):
        
            # user url
            user_url = one_review[i].find('div', attrs = {'class':'_23Rml'}).find('a').get('href')
            print('user_url = '+user_url)
            
            # user url로부터 user code 뽑아내기
            user_code = re.findall(r"my/(\w+)", user_url)[0]
            print('user_code = '+user_code)
            
            # review 1개에 대한 id 만들기 
            res_code = re.findall(r"restaurant/(\d+)", thisurl)[0]
            review_id = str(res_code)+"_"+user_code
            print('review_id = '+review_id)
            
            # rating, 별점
            rating = one_review[i].find('span', attrs = {'class':'_2tObC'}).text
            print('rating = '+rating)
            
            # 주의!!! 사진 리뷰 유무에 따라 날짜 파싱코드 다름
            # ('span', attrs = {'class':'_3WqoL'})
            # 사진 없는 경우 : 총 6개 중 4번째
            # 사진 있는 경우 : 총 5개 중 3번째
            
            # date
            # 사진 리뷰 없음
            if len(one_review[i].find_all('span', attrs = {'class':'_3WqoL'})) == 5:
                date = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[2].text
            elif len(one_review[i].find_all('span', attrs = {'class':'_3WqoL'})) == 6:
                date = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[3].text
            else:
                date = ""
            print('date = '+date)
            
            # review 내용
#             try : 
#                 review_content = one_review[i].find('span', attrs = {'class':'WoYOw'}).text
#             except: # 리뷰가 없다면
#                 review_content = ""
#             print('리뷰 내용 : '+review_content)
            
            
            # image 내용
            sliced_soup = one_review[i].find('div', attrs = {'class':'_1aFEL _2GO1Q'})

            if (sliced_soup != None):
                sliced_soup = sliced_soup.find('div',attrs={'class':'dRZ2X'})
                
                try:
                    img_url = 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src='+re.findall(r'src=(.*jpeg)', str(sliced_soup))[0]
                except :
                    if (len(re.findall(r'src=(.*jpg)', str(sliced_soup)))!= 0):
                        img_url = 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src='+re.findall(r'src=(.*jpg)', str(sliced_soup))[0]
                    elif (len(re.findall(r'src=(.*png)', str(sliced_soup)))!= 0):
                        img_url = 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src='+re.findall(r'src=(.*png)', str(sliced_soup))[0]
                    else :
                        img_url = ""
            else:
                img_url = ""
                
            print('이미지 url : '+img_url)
            print('----------------------------------------------')
            print('\n')

            
            #리뷰정보
            # user_review_id
            # user_review_id[restaurant_name][user_code] = review_id
            
            # review_json
            # review_json[restaurant_name][review_id] = review_content
            
            # image_json
            image_json[restaurant_name][review_id] = img_url
            
            # rating_df_list
            naver_review = user_code, restaurant_name, rating, date
            rating_list.append(naver_review)
            
            
    
    
    except NoSuchElementException:
            none_review = "네이버 리뷰 없음"
            print(none_review)
            review_num = 0
            #리뷰정보 = restaurant_name, restaurant_classification, review_num, none_review
            
            # rating_df_list
            naver_review = user_code, restaurant_name, none_review, none_reivew
            rating_list.append(naver_review)
            
            
    print('\n')

401번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 엘림소반
식당 구분 : 한식
----------------------------------------------
리뷰 총 개수 : 3
user_url = https://m.place.naver.com/my/5d39512841a5111e851390ba/review
user_code = 5d39512841a5111e851390ba
review_id = 38554572_5d39512841a5111e851390ba
rating = 5
date = 2021.06.11
이미지 url : 
----------------------------------------------


user_url = https://m.place.naver.com/my/5da2d44c8f87a842bcd16462/review
user_code = 5da2d44c8f87a842bcd16462
review_id = 38554572_5da2d44c8f87a842bcd16462
rating = 4
date = 2020.03.14
이미지 url : 
----------------------------------------------


user_url = https://m.place.naver.com/my/5bfa4752a8e039c37e5a1551/review
user_code = 5bfa4752a8e039c37e5a1551
review_id = 38554572_5bfa4752a8e039c37e5a1551
rating = 5
date = 2020.03.06
이미지 url : 
----------------------------------------------




402번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 석경초밥
식당 구분 : 초밥,롤
----------------------------------------------
리뷰 총 개수 : 10
user_url = https://m.place.naver.com/my/5e7

In [3]:
rating_df = pd.DataFrame(rating_list)
rating_df.columns = ['UserID','ItemID','Rating','Timestamp']
rating_df.to_csv('rating9.csv', encoding='utf-8-sig')

import json

file_path = "./user_review_id.json"
with open(file_path,'w', encoding="UTF-8") as outfile:
    json.dump(user_review_id, outfile, ensure_ascii=False)
    
file_path = "./review.json"
with open(file_path,'w', encoding="UTF-8") as outfile:
    json.dump(review_json, outfile, ensure_ascii=False)

file_path = "./image.json"
with open(file_path,'w', encoding="UTF-8") as outfile:
    json.dump(image_json, outfile, ensure_ascii=False)

NameError: name 'rating_list' is not defined

## 최종 크롤링된 데이터프레임에
## 공통 값을 기준으로(열 기준) concat하여 주소 붙이기 !!!

In [ ]:
# 수정사항
# 1. 1번파일에 상호명+도로명전체주소 처리한 새로운 열만들어서
# 2. 이 새로운 열로 검색 url키워드 만들기
# 3. 만들어진 키워드로 검색